In [201]:
import time
import requests
import os
import datetime

from rich import print
from dotenv import load_dotenv

import dspy
from dspy import (
    Predict,
    Signature,
    ChainOfThought,
    LM,
    settings
)

load_dotenv('./.env')
nai_api_key = os.getenv('NAI_API_KEY')
nai_url= 'https://ai.nutanix.com/api/v1/chat/completions'

In [178]:
question = "Produce the answer. We start with 15 eggs and if 5 are taken out how many are left? Think step by step"

### Calling the NAI-API directly to get the response

In [179]:
start_time = time.time()
response = requests.post(nai_url, 
                         headers={
                                "Authorization": f"Bearer {nai_api_key}", 
                                "accept": "application/json",
                                "Content-Type": "application/json"
                            }, 
                         json={
                                "model": "vllm-llama-3-1",
                                "messages": [
                                    {
                                        "role": "user",
                                        "content": f"{question}"
                                    }
                                ],
                                "max_tokens": 256,
                                "stream": False
                            }, 
                         verify=False)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} seconds")

if response.status_code == 200:
    print("Response from the API:")
    print(response.json())
else:
    print(f"Failed with status code: {response.status_code}")
    print("Error message:", response.text)

/Users/yashaswi.piplani/Downloads/DSPy-Python/venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ai.nutanix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Time taken: 2.9506478309631348 seconds

Response from the API:

{
    'id': '63079776-1546-4a63-8039-d5822a4ae27b',
    'object': 'chat.completion',
    'created': 1730271089,
    'model': 'vllm-llama-3-1',
    'choices': [
        {
            'index': 0,
            'message': {
                'role': 'assistant',
                'content': "Let's break it down step by step.\n\n1. We start with 15 eggs.\n2. 5 eggs are taken 
out.\n3. To find the number of eggs left, we need to subtract the number of eggs taken out (5) from the initial 
number of eggs (15).\n4. 15 - 5 = 10\n\nSo, there are 10 eggs left."
            },
            'finish_reason': 'stop',
            'content_filter_results': {
                'hate': {'filtered': False},
                'self_harm': {'filtered': False},
                'sexual': {'filtered': False},
                'violence': {'filtered': False},
                'jailbreak': {'filtered': False, 'detected': False},
                'profanity': {'filtered': False, 'detected': False}
            }
        }
    ],
    'usage': {
        'prompt_tokens': 63,
        'completion_tokens': 80,
        'total_tokens': 143,
        'prompt_tokens_details': None,
        'completion_tokens_details': None
    },
    'system_fingerprint': ''
}

In [180]:
print(response.json()['choices'][0]['message']['content'])

Let's break it down step by step.

1. We start with 15 eggs.
2. 5 eggs are taken out.
3. To find the number of eggs left, we need to subtract the number of eggs taken out (5) from the initial number of
eggs (15).
4. 15 - 5 = 10

So, there are 10 eggs left.

---
### Using the LM class to create a LM object of the model from NAI-API and configuring the same to use for inference via DSPy Modules

In [181]:
class CustomInferenceEngineModel(LM):
    def __init__(self, **kwargs):
        self.endpoint_url = nai_url
        self.api_key = nai_api_key
        self.kwargs = {
            "temperature": kwargs.get("temperature"),
        }
        self.cache = None,
        self.model_type = "chat"
        self.model = "huggingface/meta-llama/Meta-Llama-3.1-8B-Instruct"
        self.history = []

    def predict(self):
        headers={
                "Authorization": f"Bearer {self.api_key}", 
                "accept": "application/json",
                "Content-Type": "application/json"
                }
            
        request_payload = {
                            "model": "vllm-llama-3-1",
                            "messages": [
                                {
                                    "role": "user",
                                }
                            ],
                            "max_tokens": 256,
                            "stream": False,
                         }
        
        response = requests.post(
            self.endpoint_url,
            json=request_payload,
            headers=headers
        )
        
        self.history.append({
            "response": response.json(),
            "timestamp": datetime.now().isoformat()
        })
        return response.json()

In [182]:
my_model = CustomInferenceEngineModel()
settings.lm = my_model

In [183]:
pred_mod = Predict('Question -> Answer')
response = pred_mod(Question=question)
print(response.Answer)

First, we start with 15 eggs.
Then, 5 eggs are taken out.
To find out how many eggs are left, we subtract 5 from 15.
15 - 5 = 10
So, there are 10 eggs left.

In [184]:
my_model.history

[{'prompt': None,
  'messages': [{'role': 'system',
    'content': 'Your input fields are:\n1. `Question` (str)\n\nYour output fields are:\n1. `Answer` (str)\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## Question ## ]]\n{Question}\n\n[[ ## Answer ## ]]\n{Answer}\n\n[[ ## completed ## ]]\n\nIn adhering to this structure, your objective is: \n        Given the fields `Question`, produce the fields `Answer`.'},
   {'role': 'user',
    'content': '[[ ## Question ## ]]\nProduce the answer. We start with 15 eggs and if 5 are taken out how many are left? Think step by step\n\nRespond with the corresponding output fields, starting with the field `Answer`, and then ending with the marker for `completed`.'}],
  'kwargs': {'temperature': None},
  'response': ModelResponse(id='chatcmpl-1b3e44cb-8010-47d1-889a-ade36ea00d19', choices=[Choices(finish_reason='stop', index=0, message=Message(content='[[ ## Answer ## ]]\nFirst, we start with

---
### Developing custom signature

In [185]:
from dspy import (
    InputField,
    OutputField
)

class MultiClass(Signature):
    """Classify the given data into Address, Human's name, Location, Building, Amount"""
    sentence = InputField(desc="Data to be classified")
    data_type = OutputField(desc="Falls in one of the catagories")

In [186]:
pred_class = Predict(MultiClass)
print(pred_class)

Predict(MultiClass(sentence -> data_type
    instructions="Classify the given data into Address, Human's name, Location, Building, Amount"
    sentence = Field(annotation=str required=True json_schema_extra={'desc': 'Data to be classified', 
'__dspy_field_type': 'input', 'prefix': 'Sentence:'})
    data_type = Field(annotation=str required=True json_schema_extra={'desc': 'Falls in one of the catagories', 
'__dspy_field_type': 'output', 'prefix': 'Data Type:'})
))

In [187]:
get_json_data = Predict('required_data -> json_output')

prompt = "Provide one example of Address, Human's name, Location, Building, Amount"

resp = get_json_data(required_data=prompt)
print(resp)

Prediction(
    json_output='{\n  "Address": "123 Main St",\n  "Human\'s name": "John Doe",\n  "Location": "New York",\n  
"Building": "Empire State Building",\n  "Amount": 1000\n}'
)

In [188]:
import json

recd_data = json.loads(resp.json_output)
recd_data

{'Address': '123 Main St',
 "Human's name": 'John Doe',
 'Location': 'New York',
 'Building': 'Empire State Building',
 'Amount': 1000}

In [189]:
for vals in recd_data.values():
    print("Classifying: ", vals)
    class_resp = pred_class(sentence=vals)
    print("Output class is: ", class_resp.data_type)

Classifying:  123 Main St

Output class is:  Address

Classifying:  John Doe

Output class is:  Human's name

Classifying:  New York

Output class is:  Location

Classifying:  Empire State Building

Output class is:  Building

Classifying:  1000

Output class is:  Amount

---
### Using ChainOfThought Module

In [253]:
question2 = "Provide me the top 2 countries with their GDP for last 3 years"

In [254]:
cot_qa = ChainOfThought("question -> reasoning, answer_as_json", n=2)
cot_response = cot_qa(question=question2)
print(cot_response.completions.answer_as_json[0])
print(cot_response.completions.answer_as_json[1])

{
    "countries": [
        {"name": "United States", "gdp_2018": 22.67, "gdp_2019":

{
  "top2Countries": [
    {
      "country": "United States",
      "gdp2020": 22.67,
      "gdp2021": 23.32,

In [256]:
my_model.inspect_history(n=3)





[2024-10-30T12:31:59.462540]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer_as_json` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer_as_json ## ]]
{answer_as_json}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer_as_json`.


User message:

[[ ## question ## ]]
Provide me the top 2 countries and their GDP for last 3 years

Respond with the corresponding output fields, starting with the field `reasoning`, then `answer_as_json`, and then ending with the marker for `completed`.


Response:

[[ ## reasoning ## ]]
To provide the top 2 countries and their GDP for the last 3 years, I can use a dataset of global GDP data from 2020, 2021, and 2022. I will first sort the countries by their 2022 GDP 